In [ ]:
from sqlalchemy import create_engine, text, inspect
from docx import Document
import pandas as pd

## Get data in a database

In [ ]:
# Create a connection string and set up the database connection
db_uri = "sqlite:///requirements.db"
engine = create_engine(db_uri)

# Read the requirements from an Excel file 
df = pd.read_excel("src_data.xlsx")

# Store the data in the database - note we "replace" the content for now
num_entries = df.to_sql("requirements", engine.connect(), index=False, if_exists='replace')
print(f"There {num_entries} added to the database")

## Get metadata

Let's check if a table was created and what the columns are

In [ ]:
# create an inspector object - this can be used to get metadata of our "engine"
inspector = inspect(engine)

# get and print de tables in the database
tables = inspector.get_table_names()
print(tables)

# get and print the columns of a specific tabe
cols = inspector.get_columns(tables[0])
for col in cols:
    print(col)

## Query the database

Get all rows of the table

#### DOES EACH REQUIREMENT ONLY RELATES TO ONE TEAM?
- if so we only need one colum and not two like we have now

In [ ]:
# Define your SQL query
sql_query = text("SELECT * FROM requirements")
pd.read_sql(sql_query, engine.connect()) 

## Add requirements to MS Word and save

In [ ]:
# create a word document object
document = Document()

# Iterate over the pandas dataframe rows and insert the requirements in the
# word document
for idx, row in df.iterrows():
    requirement_id = row['number']
    requirement_desc =row['requirement']
    document.add_paragraph(f"Requirement ID: {requirement_id}")
    document.add_paragraph(f"description: {requirement_desc}")
    document.add_paragraph("")
    
# Save the populated document
document.save("output_document.docx")

## Add requirements to MS Word as table

Below is a very simple working example that writes contents of a database to a word table 

In [ ]:
from sqlalchemy import create_engine, text
from docx import Document
from docx.enum.table import WD_TABLE_ALIGNMENT

# Set up the database connection
db_uri = "sqlite:///requirements.db"
engine = create_engine(db_uri)

# Define your SQL query
sql_query = text("SELECT * FROM requirements")

# Execute the query
with engine.connect() as connection:
    requirements = connection.execute(sql_query)

    # Create a new Word document
    document = Document()

    # Create a table to display the requirements
    table = document.add_table(rows=1, cols=2)
    table.style = 'Table Grid'  # Apply a table style

    # Set table alignment to center
    table.alignment = WD_TABLE_ALIGNMENT.CENTER

    # Add table headers
    headers = table.rows[0].cells
    headers[0].text = 'Requirement ID'
    headers[1].text = 'Description'
    
    # Iterate over the requirements and populate the table
    for requirement in requirements:
        requirement_id, description, phase, mv, md = requirement
        row = table.add_row().cells
        row[0].text = str(requirement_id)
        row[1].text = description

    # Save the populated document
    document.save("output_document.docx")

## More realistic approach

In general requirements can be depend at different parts of the lifecycle (e.g. Initiation, Development, Use, Retirement).
In the example database each requirement has a field to state to which phase it belongs. We want to have a table per phase in a seperate document for each team.

In [ ]:
from sqlalchemy import create_engine, text
from docx import Document
from docx.enum.table import WD_TABLE_ALIGNMENT, WD_CELL_VERTICAL_ALIGNMENT
from docx.shared import Inches, Mm

def set_column_width(table,column,width_mm):
    table.allow_autofit = False
    for row in table.rows:
        row.cells[column].width = Mm(width_mm)

# get a list of model life cycle phases
stmt = text("SELECT DISTINCT life_cycle_phase FROM requirements")
life_cycle_phases = pd.read_sql(stmt, engine).values.squeeze().tolist()

# TODO
# We need some outer loop to go create a separate doc for md and mv

# create a MS Word object
document = Document()

for cycle in life_cycle_phases:
    # Read all the data in for a specific phase in the life cycle
    stmt = text(f"SELECT * FROM requirements WHERE life_cycle_phase = '{cycle}' ")
    df = pd.read_sql(stmt, engine)

    # Create a table to display the requirements
    # adjust the cols argument to allign with the num columns 
    document.add_paragraph(f"Category: {cycle}")
    table = document.add_table(rows=1, cols=2)
    table.columns[1].width = 1000000
    table.style = 'Table Grid'  # Apply a table style
    
    # Set table alignment to center
    table.alignment = WD_TABLE_ALIGNMENT.CENTER

    # Add table headers
    headers = table.rows[0].cells
    headers[0].text = 'Requirement ID'
    headers[1].text = 'Description'
    
    for idx, row in df.iterrows():
        table_row = table.add_row().cells
        table_row[0].text = str(row['number'])
        table_row[1].text = row['requirement']
        
    document.add_paragraph("")    
    
# Save the populated document
document.save("output_document.docx")

In [ ]:
table.columns[0].width